In [ ]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0])+'/dash_app')
    __package__ = "dash_app"

In [ ]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from settings import *
from charts import stations_map,map_columns

In [ ]:
STATION_COLORS = {
    'Buoy': '#e69800',
    'Synoptic': '#ff0000',
    'Rainfall': '#ffff00',
    'Climate': '#004da8',
    'WaveRide/SmartBayObsCenter': '#ffff00',
    'MI_Survey': '#ff0000',
    'EPA': '#002673'
}

MAP_LAYOUT = dict(
    legend=dict(title='<b>Station Type</b>',
                x=0.01),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=400,
    margin=dict(t=0, b=0, r=0, l=0),
    mapbox=dict(bearing=0,
                center=dict(
                    lat=53.4,
                    lon=352
                ),
                pitch=0,
                zoom=5,
                style="open-street-map"  # does not require token
                )
)

def stations_map(df):

    def stations_map_hovertemplate(df):
        return ['Name: '+'{}<br>'.format(n)+'
                'Type: '+'{}<br>'.format(t)+
                'Station No.: '+'{}<br>'.format(sN)+
                'County.: '+'{}<br>'.format(cnty)+
                'Open Year.: '+'{}<br>'.format(oY)+
                'Height: '+'{:.2f} m<br>'.format(h)+
                # 'Easting: '+'{}'.format(easting)+'<br>' +
                # 'Northing: '+'{}'.format(northing)+'<br>'
                'Lat: '+'{:.2f} \u00b0<br>'.format(lat)+
                'Lon: '+'{:.2f} \u00b0<br>'.format(lon)+
                '<extra></extra>'
                for n, t, sN, cnty, oY, cY, h, lat, lon in zip(list(df['name']),
                                                                            list(
                    df['Type']),
            list(
                    df['Station_Nu']),
            list(
                    df['County']),
            list(
                    df['Open_Year']),
            list(
                    df['Close_Year']),
            list(
                    df['Height__m_']),
            # list(
            #         df['Easting']),
            # list(
            #         df['Northing']),
            list(
                    df['Latitude']),
            list(
                    df['Longitude']),
        )]

    buoyDF = df.loc[df['Type'] == 'Buoy']
    synopticDF = df.loc[df['Type'] == 'Synoptic']
    rainfallDF = df.loc[df['Type'] == 'Rainfall']
    climateDF = df.loc[df['Type'] == 'Climate']
    tidbiTDF = df.loc[df['Type'] == 'TidbiT Sea Temperature Station']

    tidbiTTrend = go.Scattermapbox(
        name='tidbiT',
        lon=tidbiTDF.Longitude,
        lat=tidbiTDF.Latitude,
        marker=dict(color=STATION_COLORS['Buoy'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(tidbiTDF),
    )

    buoyTrend = go.Scattermapbox(
        name='Buoys',
        lon=buoyDF.Longitude,
        lat=buoyDF.Latitude,
        marker=dict(color=STATION_COLORS['Buoy'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(buoyDF),
    )
    rainfallTrend = go.Scattermapbox(
        name='Rainfall',
        lon=rainfallDF.Longitude,
        lat=rainfallDF.Latitude,
        marker=dict(color=STATION_COLORS['Rainfall'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(rainfallDF),
    )

    synopticTrend = go.Scattermapbox(
        name='Synoptic',
        lon=synopticDF.Longitude,
        lat=synopticDF.Latitude,
        marker=dict(color=STATION_COLORS['Synoptic'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(synopticDF),
    )

    climateTrend = go.Scattermapbox(
        name='Climate',
        lon=climateDF.Longitude,
        lat=climateDF.Latitude,
        marker=dict(color=STATION_COLORS['Climate'],
                    size=7),
        hovertemplate=stations_map_hovertemplate(climateDF),
    )
    stations_map = go.Figure(
        data=[buoyTrend, synopticTrend, rainfallTrend, climateTrend,tidbiTTrend],
        layout=MAP_LAYOUT)
    
    return stations_map



In [ ]:
"""
Sea surface temp
"""
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.1/'
data_csv = data_path + 'Figure3.1_data.csv'
"""
Tidy data for charts
"""
xls = pd.ExcelFile(
    data_path+'MeanAnnualSeaSurfaceTemperature_Anomalies_MalinHead.xlsx')
original_df = pd.read_excel(xls, 'MALIN_Timeseries',skiprows=[])
original_df
columns_dict = {'datetime':'year',
               'mean__annual__surface_sea_temperature':'temperature',
               'mean__annual__surface_sea_temperature_anomaly':'Calculated Anomalies',
               'normal_1981_2010__surface_sea_temperature':'Unnamed: 5',
               'moving_average__5year__surface_sea_temperature_anomaly':'5 year moving average',
                  }
df = map_columns(columns_dict, original_df)
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y')
df['moving_average__5year__surface_sea_temperature']=(
    df['moving_average__5year__surface_sea_temperature_anomaly']+
    df['normal_1981_2010__surface_sea_temperature'])
df['location'] = 'MalinHead'
df
df.to_csv(data_csv)
# df[0:20]

In [ ]:
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.1/'
data_csv = data_path + 'Figure3.1_data.csv'
df = pd.read_csv(data_csv, index_col=0)
# movingAverageTotals = df.temperature.rolling(window=5, center=True).mean() 

annualTrace = go.Bar(x=df["datetime"],
                    y=df["mean__annual__surface_sea_temperature_anomaly"],
                    text=df["mean__annual__surface_sea_temperature"],
                    name='Annual',
                    marker=dict(
                            # color="#214a7b", color used in report
                            color=TIMESERIES_COLOR_2,
                            opacity=0.5
                            ),
                    hovertemplate='%{x|%Y}<br>' +
                            '<b>Annual</b><br>' +
                            'Mean: %{text:.2f} \u00b0C<br>' +
                            'Anomaly: %{y:.2f} \u00b0C<extra></extra>'
                            )
movingAverage = go.Scatter(x=df["datetime"],
                    y=df["moving_average__5year__surface_sea_temperature_anomaly"],
                    text=df["moving_average__5year__surface_sea_temperature"],
                    name='5 Year Moving Average',
                    mode='lines',  # 'line' is default
                    line_shape='spline',
                    line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_1,
                            width=2),
                    hovertemplate='%{x|%Y}<br>' +
                            '<b>5 Year Moving Average</b><br>' +
                            'Mean: %{text:.2f} \u00b0C<br>' +
                            'Anomaly: %{y:.2f} \u00b0C<extra></extra>'
                            )
normal = go.Scatter(x=df["datetime"],
                    y=df["normal_1981_2010__surface_sea_temperature"],
                    name='1981-2010 Average',
                    mode='lines',  # 'line' is default
                    line_shape='spline',
                    line=dict(color=TIMESERIES_COLOR_3, #color used in report
                            width=1),
                    hoverinfo='skip',
                            )

figure_3_1 = make_subplots(specs=[[{'secondary_y': True}]])
figure_3_1.add_trace(annualTrace,
            secondary_y=False,)
figure_3_1.add_trace(movingAverage,
            secondary_y=False,)
figure_3_1.add_trace(normal,
            secondary_y=True,)
figure_3_1.update_layout(TIMESERIES_LAYOUT)
figure_3_1.update_yaxes(title_text='Difference (\u00b0C) from 1981-2010 Average',
                        secondary_y=False,
                        range=[-1, 1],
                        showgrid=False,
                        dtick=0.25,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_2
                        )
figure_3_1.update_yaxes(title_text='Sea Surface Temperature (\u00b0C)',
                    secondary_y=True,
                    range=[9.5, 11.6],
                    showgrid=False,
                    dtick=0.5,  # dtick sets the distance between ticks
                    tick0=10.6,  # tick0 sets a point to map the other ticks
                    fixedrange=True,
                    )
figure_3_1.update_xaxes(
    title_text='Year',
    fixedrange=True,
    showspikes=True,  
    spikethickness=2, 
    ) 

figure_3_1

In [ ]:
"""
Sea subsurface temp
"""
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.3/'
data_csv = data_path + 'Figure3.3_data.csv'
"""
Tidy data for charts
"""
# xls = pd.ExcelFile(
#     data_path+'SubSurfaceTemperature_Anomalies_Rockall.xlsx')
# original_df = pd.read_excel(xls, 'Depth_Rockall',skiprows=[20])
# original_df
xls = pd.ExcelFile(
    data_path+'SubSurfaceTemperature_Anomalies_Rockall.xlsx')
original_df = xls.parse('Depth_Rockall', skiprows=20, index_col=None, na_values=['NA'])
columns_dict = {'datetime':'Decimal Year',
               'mean__annual__subsurface_sea_temperature':'Temperature °C',
               'mean__annual__subsurface_sea_temperature_anomaly':'Temperature Anomaly °C',
               'normal_1981_2010__surface_sea_temperature':'1981-2010 (Normalized)',
               'moving_average__5year__surface_sea_temperature_anomaly':'5 year moving average',
                  }
df = map_columns(columns_dict, original_df)
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y')
df['moving_average__5year__surface_sea_temperature']=(
    df['moving_average__5year__surface_sea_temperature_anomaly']+
    df['normal_1981_2010__surface_sea_temperature'])
df['location'] = 'RockallTrough'
df
# df.to_csv(data_csv)
# df[0:20]

In [ ]:
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.3/'
data_csv = data_path + 'Figure3.3_data.csv'
df = pd.read_csv(data_csv, index_col=0)
annualTrace = go.Scatter(x=df["datetime"],
                 y=df["mean__annual__subsurface_sea_temperature"],
                 name='Annual Mean',
                  mode='lines',  # 'line' is default
                    line_shape='spline',
                    line=dict(
                            color=TIMESERIES_COLOR_2,
                            width=2),
                 hovertemplate='%{x|%Y}<br>' +
                        '<b>Annual Mean</b><br>' +
                        'Temperature: %{y:.2f} \u00b0C<extra></extra>'
                        )
movingAverage = go.Scatter(x=df["datetime"],
                    y=df["moving_average__5year__surface_sea_temperature"],
                    name='5 Year Moving Average',
                    mode='lines',  # 'line' is default
                    line_shape='spline',
                    line=dict(
                            color=TIMESERIES_COLOR_1,
                            width=2),
                    hovertemplate='%{x|%Y}<br>' +
                            '<b>5 Year Moving Average</b><br>' +
                            'Temperature: %{y:.2f} \u00b0C<extra></extra>'
                            )

figure_3_3 = go.Figure(data=[annualTrace, movingAverage], layout=TIMESERIES_LAYOUT)
figure_3_3.update_yaxes(title_text='Sea Subsurface Temperature (\u00b0C)',
                        fixedrange=True,
                        )

figure_3_3.update_xaxes(
    title_text='Year',
    fixedrange=True, 
    showspikes=True,  
    spikethickness=2, 
) 

figure_3_3

In [ ]:
"""
Sea surface temp
Read from excel, no longer used
"""

data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.1/'
xls = pd.ExcelFile(
    data_path+'MeanAnnualSeaSurfaceTemperature_Anomalies_MalinHead.xlsx')
df = pd.read_excel(xls, 'MALIN_Timeseries')
movingAverageTotals = df.temperature.rolling(window=5, center=True).mean() 
df["5 Year Moving Average - Mean"]=movingAverageTotals
df

In [ ]:

annualTrace = go.Bar(x=df["year"],
                     y=df["Calculated Anomalies"],
                     text=df["temperature"],
                     name='Annual',
                     marker=dict(
                            # color="#214a7b", color used in report
                            color=TIMESERIES_COLOR_2,
                            opacity=0.5
                            ),
                     hovertemplate='%{x}<br>' +
                            '<b>Annual</b><br>' +
                            'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
movingAverage = go.Scatter(x=df["year"],
                     y=df["5 year moving average"],
                     text=df["5 Year Moving Average - Mean"],
                     name='5yr Moving Average',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_1,
                            width=2),
                     hovertemplate='%{x}<br>' +
                            '<b>5yr Moving Average</b><br>' +
                            'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
normal = go.Scatter(x=df["year"],
                     y=df["Unnamed: 5"],
                     name='1981-2010 Normal',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(color="#fdbf2d", #color used in report
                               width=1),
                     hoverinfo='skip',
                            )



In [ ]:
figure_3_1 = make_subplots(specs=[[{'secondary_y': True}]])
figure_3_1.add_trace(annualTrace,
            secondary_y=False,)
figure_3_1.add_trace(movingAverage,
            secondary_y=False,)
figure_3_1.add_trace(normal,
            secondary_y=True,)

figure_3_1.update_layout(TIMESERIES_LAYOUT)
figure_3_1.update_yaxes(title_text='Difference (°C) from 1981-2010 Normal',
                        secondary_y=False,
                        range=[-1, 1],
                        showgrid=False,
                        dtick=0.25,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_2
                        )
figure_3_1.update_yaxes(title_text='Annual SST (°C)',
                        secondary_y=True,
                        range=[9.5, 11.6],
                        showgrid=False,
                        dtick=0.5,  # dtick sets the distance between ticks
                        tick0=10.6,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        )

figure_3_1.update_xaxes(
    title='Year',
    fixedrange=True,
    tickformat='000',  
    showspikes=True,  
    spikethickness=2, 
) 


In [ ]:
"""
Sea subsurface temp
Read from Excel, not used
"""
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.3/'
xls = pd.ExcelFile(
    data_path+'SubSurfaceTemperature_Anomalies_Rockall.xlsx')
df = xls.parse('Depth_Rockall', skiprows=20, index_col=None, na_values=['NA'])
# df = pd.read_excel(xls, 'Depth_Rockall')
# movingAverageTotals = df.temperature.rolling(window=5, center=True).mean() 
# df["5 Year Moving Average - Mean"]=movingAverageTotals
df

In [ ]:
annualTrace = go.Bar(x=df["Decimal Year"],
                     y=df["Temperature Anomaly °C"],
                     text=df["Temperature °C"],
                     name='Annual',
                     marker=dict(
                            # color="#214a7b", color used in report
                            color=TIMESERIES_COLOR_2,
                            opacity=0.5
                            ),
                     hovertemplate='%{x}<br>' +
                            '<b>Annual</b><br>' +
                            'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
movingAverage = go.Scatter(x=df["Decimal Year"],
                     y=df["5 year moving average"],
                    #  text=df["5 Year Moving Average - Mean"],
                     name='5yr Moving Average',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_1,
                            width=2),
                     hovertemplate='%{x}<br>' +
                            '<b>5yr Moving Average</b><br>' +
                            # 'Total: %{text:.2f} °C<br>' +
                            'Anomaly: %{y:.2f} °C<extra></extra>'
                            )
normal = go.Scatter(x=df["Decimal Year"],
                     y=df["1981-2010 (Normalized)"],
                     name='1981-2010 Normal',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(color="#fdbf2d", #color used in report
                               width=1),
                     hoverinfo='skip',
                            )


In [ ]:
figure_3_3 = make_subplots(specs=[[{'secondary_y': True}]])
figure_3_3.add_trace(annualTrace,
            secondary_y=False,)
figure_3_3.add_trace(movingAverage,
            secondary_y=False,)
figure_3_3.add_trace(normal,
            secondary_y=True,)

figure_3_3.update_layout(TIMESERIES_LAYOUT)
figure_3_3.update_yaxes(title_text='Difference (°C) from 1981-2010 Normal',
                        secondary_y=False,
                        range=[-0.25, 0.45],
                        showgrid=False,
                        dtick=0.05,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_2
                        )
figure_3_3.update_yaxes(title_text='Annual SST (°C)',
                        secondary_y=True,
                        range=[3.93, 4.59],
                        showgrid=False,
                        dtick=0.1,  # dtick sets the distance between ticks
                        tick0=4.17,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        )

figure_3_3.update_xaxes(
    title='Year',
    fixedrange=True,
    tickformat='000',  
    showspikes=True,  
    spikethickness=2, 
) 


In [ ]:
"""
Infrastructure Map
"""
DATA_PATH="/Users/dan/OneDrive - University College Cork/Status_Tool/"
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Map3.1/'
stationsDF = pd.read_csv(
            data_path+'Map3.1_StationTable_MI.txt')

rockallDF = pd.read_csv(
            data_path+'Map3.1_StationTable_RockallTroughSection.txt')
ellettLineDF = pd.read_csv(
            data_path+'Map3.1_StationTable_ExtendedEllettLineBuoy.txt')
tidbiDF = pd.read_csv(
            data_path+'Map3.1_StationTable_TidbiT.txt')
stationsDF 

In [ ]:
stationsDF_subsurface = stationsDF[stationsDF['name'].isin(['M6_Buoy', 'SmartBay Wave Buoy'])]
stationsDF_subsurface

In [ ]:
rockallDF.columns

In [ ]:
ellettLineDF

In [ ]:
rockallDF

In [ ]:
tidbiDFNew=pd.DataFrame(columns=rockallDF.columns)
tidbiDFNew['name']=tidbiDF["localId"]
tidbiDFNew['Latitude']=tidbiDF["latitude"]
tidbiDFNew['Longitude']=tidbiDF["longitude"]
tidbiDFNew['Open_Year']=pd.to_datetime(tidbiDF["beginLifes"]).dt.year
tidbiDFNew['Close_Year']=pd.to_datetime(tidbiDF["endLifespa"]).dt.year
tidbiDFNew['Type']=tidbiDF["datasetNam"]
tidbiDFNew

In [ ]:
combinedDF = pd.concat([stationsDF,ellettLineDF,tidbiDFNew])
combinedDF.reset_index
combinedDF

In [ ]:
    map_3_1=stations_map(combinedDF)
    map_3_1.update_layout(
    mapbox=dict(bearing=0,
            center=dict(
                lat=56,
                lon=346
            ),
            pitch=0,
            zoom=3.8))
    map_3_1